## 1.One-hot核心问题：它不会“表达语义”，只会表达“身份”
>设词表的大小|V|=100000,one-hot向量是10万维，但只有一个位置是1
### (1)相似性完全消失
- 猫 和 狗 的 one-hot 内积 = 0

- 猫 和 苹果 的 one-hot 内积也 = 0

⇒ 在 one-hot 空间里，“猫≈狗”这类语义相近关系根本不存在（所有不同词都等距/等不相似）。

### (2)稀疏 + 维度灾难 + 泛化弱
- 维度随词表线性增长，向量极稀疏

- 任何依赖距离/角度的模型都学不到“相近词更近”的结构

- 你在训练集中没怎么见过的词（低频词），更难泛化，因为表示里没有“可迁移的共享结构”

### (3)参数量容易爆
如果用 one-hot 直接接一个线性层得到隐藏层 $h \in \mathbb{R}^{d}$:
- 权重矩阵 $W \in \mathbb{R}^{|V| \times d}$

- 计算 one-hot @ W 本质上就是查表：取出第 i 行作为词向量

⇒ 这说明：embedding 层就是把 one-hot 变成“可学习的稠密向量”，并且让相似词能学到相似向量。

## 2.Distributional Hypothesis: 语义来自上下文
>一个词的语义由它常出现的上下文分布来决定。如果两个词在大量语境中“可互换/上下文相似”，它们的语义就相似。

把它落实到可计算的东西上就是：
- 统计“词w”附近窗口内出现“上下文词 c”的频次

- 得到每个词的“上下文分布向量”

- 分布相近 ⇒ 语义相近

## 3.共现矩阵：把”语义来自上下文“变成一个可计算的矩阵
共现矩阵 $X \in \mathbb{R}^{|V| \times|V|}$；
- 行：中心词w

- 列：上下文词c

- Xw,c：在窗口内c出现在w周围的次数

## 4.共现矩阵示例

### (1)语料
1.我 喜欢 自然 语言 处理

2.我 喜欢 深度 学习

3.深度 学习 改变 世界

### (2)此表顺序
['世界','喜欢','处理','学习','我','改变','深度','自然','语言']

### (3)共现矩阵（行=中心词，列=上下文词）
|center\context|世界|喜欢|处理|学习|我|改变|深度|自然|语言|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|世界|0|0|0|0|0|1|0|0|0|
|喜欢|0|0|0|0|2|0|1|1|0|
|处理|0|0|0|0|0|0|0|0|1|
|学习|0|0|0|0|0|1|2|0|0|
|我|0|2|0|0|0|0|0|0|0|
|改变|1|0|0|1|0|0|0|0|0|
|深度|0|1|0|2|0|0|0|0|0|
|自然|0|1|0|0|0|0|0|0|1|
|语言|0|0|1|0|0|0|0|1|0|

## 5.代码任务：构建共现矩阵

In [1]:
from collections import defaultdict
import numpy as np

corpus = [
    "我 喜欢 自然 语言 处理".split(),
    "我 喜欢 深度 学习".split(),
    "深度 学习 改变 世界".split()
]

# 1) vocab
vocab = sorted(set(w for sent in corpus for w in sent))
word2idx = {w:i for i,w in enumerate(vocab)}
V = len(vocab)

# 2) co-occurrence matrix
window = 1
X = np.zeros((V, V), dtype=int)

for sent in corpus:
    n = len(sent)
    for i, center in enumerate(sent):
        left = max(0, i - window)
        right = min(n, i + window + 1)
        for j in range(left, right):
            if j == i:
                continue
            context = sent[j]
            X[word2idx[center], word2idx[context]] += 1

print("vocab =", vocab)
print(X)


vocab = ['世界', '喜欢', '处理', '学习', '我', '改变', '深度', '自然', '语言']
[[0 0 0 0 0 1 0 0 0]
 [0 0 0 0 2 0 1 1 0]
 [0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 2 0 0]
 [0 2 0 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0]
 [0 1 0 2 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 1 0]]


## 6.从共现矩阵到Word2Vec

- 共现矩阵虽然有语义，但是太大太稀疏（|V| x |V|级别，不好存不好学）
- Word2Vec 的目标：学一个低维向量 $e_{w} \in \mathbb{R}^{d}(d \ll|V|)$ ，让它能预测上下文（Skip-gram）或被上下文预测（CBOW）.

## 7.Word2Vec
>Word2Vec 是一种用于将单词表示为向量的算法，它通过对大量文本进行训练，学习词语之间的语义关系，并将每个词表示为一个向量。Word2Vec 模型的核心目标是将词语转换为密集的、低维度的向量，以便计算机能够更好地理解词语的语义和它们之间的关系。

### 原理

Word2Vec 的基本原理是通过大量的文本数据，学习每个词在上下文中的分布式表示。它基于假设：如果两个词在语境中经常一起出现，那么它们的语义是相关的。

Word2Vec 有两个主要的模型：

>1.CBOW (Continuous Bag of Words)：给定上下文词预测目标词。其目标是根据上下文词来预测目标词。

>2.Skip-gram：给定一个目标词，预测上下文词。该模型的目标是通过目标词来预测其周围的上下文词。

### 数学原理

>CBOW 模型：

给定上下文词w1,w2,...,wn, 预测目标词wt。目标是通过最大化条件概率 $P\left(w_{t} \mid w_{1}, w_{2}, \ldots, w_{n}\right)$ 来训练模型。

目标函数为最大化每个单词的条件概率：
$$ J(\theta)=\sum_{t=1}^{T} \log P\left(w_{t} \mid w_{t-n}, \ldots, w_{t-1}, w_{t+1}, \ldots, w_{t+n}\right)$$
其中T为词汇表大小，n为窗口大小。

>Skip-gram 模型：

给定一个目标词wt，并预测上下文词w1,w2,...,wn。

目标是最大化目标词与上下文词的联合概率：
$$ P\left(w_{\text {context }} \mid w_{t}\right)=\prod_{c \in \text { context }} P\left(w_{c} \mid w_{t}\right)$$
这里的目标是通过学习使得预测的上下文词的概率最大化。

### 代码实现
实现 Word2Vec 的代码可以使用 Python 中的 gensim 库，它提供了高效的实现。以下是一个简单的代码示例：

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
import seaborn as sns
import pandas as pd

# ==========================================
# 1. 准备数据 (模拟语料)
# ==========================================
# 为了演示效果，我们手动构造一些有明显语义聚类的句子
# 分为三类：AI技术、水果、家庭关系
corpus = [
    # AI / NLP 相关
    ['machine', 'learning', 'is', 'artificial', 'intelligence'],
    ['deep', 'learning', 'uses', 'neural', 'networks'],
    ['natural', 'language', 'processing', 'is', 'nlp'],
    ['word', 'embedding', 'vector', 'space'],
    ['skip-gram', 'model', 'predicts', 'context'],
    ['algorithm', 'data', 'science', 'computer'],
    ['gpu', 'trains', 'models', 'faster'],
    
    # 水果/食物 相关
    ['apple', 'banana', 'orange', 'fruit'],
    ['i', 'eat', 'apple', 'and', 'orange'],
    ['banana', 'is', 'yellow', 'fruit'],
    ['cherry', 'strawberry', 'sweet', 'food'],
    ['lunch', 'dinner', 'breakfast', 'eat'],
    
    # 家庭/人称 相关
    ['king', 'queen', 'prince', 'princess'],
    ['father', 'mother', 'son', 'daughter'],
    ['man', 'woman', 'boy', 'girl'],
    ['brother', 'sister', 'family', 'home']
]

# 增加数据量以强化共现关系 (简单复制)
training_data = corpus * 100

print(f"语料加载完毕，共 {len(training_data)} 个句子。")

# ==========================================
# 2. 训练 Word2Vec (Skip-gram)
# ==========================================
# 参数解析：
# vector_size: 词向量维度 (通常 100-300，演示用 10)
# window: 上下文窗口大小
# min_count: 忽略出现次数过少的词
# sg: 1 表示 Skip-gram (默认0是CBOW) -> 今天的重点
# negative: 负采样个数 (通常 5-20) -> 今天的重点
# epochs: 迭代次数
model = Word2Vec(sentences=training_data, 
                 vector_size=10, 
                 window=3, 
                 min_count=1, 
                 sg=1,  # <--- 重点：开启 Skip-gram
                 negative=5, # <--- 重点：负采样
                 epochs=50,
                 seed=42)

print("\nWord2Vec 模型训练完成。")

# ==========================================
# 3. 验证训练结果
# ==========================================
def check_similarity(word):
    try:
        sims = model.wv.most_similar(word, topn=3)
        print(f"\n与 '{word}' 最相似的词:")
        for w, s in sims:
            print(f"  - {w}: {s:.4f}")
    except KeyError:
        print(f"'{word}' 不在词表中。")

check_similarity('apple')
check_similarity('learning')
check_similarity('king')

# ==========================================
# 4. 可视化 (PCA 降维)
# ==========================================
def plot_embeddings(model):
    # 获取所有词向量
    words = list(model.wv.index_to_key)
    vectors = model.wv[words]
    
    # 使用 PCA 将 10维 降到 2维 以便绘图
    pca = PCA(n_components=2)
    result = pca.fit_transform(vectors)
    
    # 创建 DataFrame 方便绘图
    df = pd.DataFrame(result, columns=['x', 'y'])
    df['word'] = words
    
    # 绘图设置
    plt.figure(figsize=(12, 10))
    sns.set_style("whitegrid")
    
    # 绘制散点
    sns.scatterplot(data=df, x='x', y='y', s=100, color='blue', alpha=0.6)
    
    # 添加标签
    for i, line in df.iterrows():
        plt.text(line['x']+0.02, line['y'], line['word'], fontsize=12)
        
    plt.title('Word2Vec (Skip-gram) Embeddings Visualization', fontsize=16)
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    
    # 保存并显示
    plt.savefig('w2v_visualization.png')
    print("\n可视化图片已保存为 'w2v_visualization.png'")
    plt.show()

print("\n开始生成可视化...")
plot_embeddings(model)

## 8.使用pytorch实现skip-gram
### (1)Skip-gram + Softmax（最原始定义）
给定中心词w，预测上下文词c：
$$P(c \mid w)=\frac{\exp \left(\mathbf{v}_{c}^{\top} \mathbf{u}_{w}\right)}{\sum_{c^{\prime} \in V} \exp \left(\mathbf{v}_{c^{\prime}}^{\top} \mathbf{u}_{w}\right)}$$

- ${u}_{w}$：中心词向量（input embedding）
- ${v}_{c}$：上下文词向量（output embedding）
- 目标：最大化所有真是共现对(w,c)的对数似然

这样做的问题是每一个中心词的分母要对全词表求和，太慢了。

### (2)负采样 Negative Simpling
对每个正样本(w,c)，采K个负样本n1,...,nk(从噪声分布采)
目标函数（对一个样本）：
$$\mathcal{L}=-\left[\log \sigma\left(\mathbf{v}_{c}^{\top} \mathbf{u}_{w}\right)+\sum_{i=1}^{K} \log \sigma\left(-\mathbf{v}_{n_{i}}^{\top} \mathbf{u}_{w}\right)\right]$$

- $\sigma(x)$: Sigmoid 函数 $\frac{1}{1+e^{-x}}$，把内积压缩到 $(0, 1)$ 之间变成概率。

- 第一部分 $\log \sigma(v_c u_w)$: 希望中心词 $u_w$ 和目标词 $v_c$ 的内积越大越好（趋近于 1）。

- 第二部分 $\sum_{i=1}^k$: 我们随机采样 $k$ 个噪音词 $w_i$（通常 $k=5 \sim 20$）。

- $\log \sigma(-v_{n_i}^T u_w)$: 这里用了个数学技巧，$\sigma(-x) = 1 - \sigma(x)$。意思是我们希望中心词 $u_w$ 和噪音词 $v_{n_i}$ 的内积越小越好（趋近于 0），也就是让 $\sigma(-x)$ 趋近于 1。

### (3)使用pytorch实现skip-gram
- 1.分词/建词表（word → id）
- 2.用滑动窗口构造训练样本对 (center,context)
- 3.两套 Embedding：in_embed 和 out_embed（对应u,v）
- 用负采样 loss 训练

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import random

# -------------------------
# 1) toy corpus
# -------------------------
corpus = [
    # AI / NLP 相关
    ['machine', 'learning', 'is', 'artificial', 'intelligence'],
    ['deep', 'learning', 'uses', 'neural', 'networks'],
    ['natural', 'language', 'processing', 'is', 'nlp'],
    ['word', 'embedding', 'vector', 'space'],
    ['skip-gram', 'model', 'predicts', 'context'],
    ['algorithm', 'data', 'science', 'computer'],
    ['gpu', 'trains', 'models', 'faster'],
    
    # 水果/食物 相关
    ['apple', 'banana', 'orange', 'fruit'],
    ['i', 'eat', 'apple', 'and', 'orange'],
    ['banana', 'is', 'yellow', 'fruit'],
    ['cherry', 'strawberry', 'sweet', 'food'],
    ['lunch', 'dinner', 'breakfast', 'eat'],
    
    # 家庭/人称 相关
    ['king', 'queen', 'prince', 'princess'],
    ['father', 'mother', 'son', 'daughter'],
    ['man', 'woman', 'boy', 'girl'],
    ['brother', 'sister', 'family', 'home']
]

# build vocab
words = [w for sent in corpus for w in sent]
cnt = Counter(words)
vocab = list(cnt.keys())
#将词转换成id
word2id = {w:i for i,w in enumerate(vocab)}
#将id转换成词
id2word = {i:w for w,i in word2id.items()}
V = len(vocab)

# -------------------------
# 2) build (center, context) pairs
# -------------------------
def build_pairs(corpus, window=2):
    pairs = []
    for sent in corpus:
        #取出每一句话词的id
        ids = [word2id[w] for w in sent]
        n = len(ids)
        #遍历句子中每一个位置i，把该位置的词当成中心词
        for i, center in enumerate(ids):
            for j in range(max(0, i-window), min(n, i+window+1)):
                if j == i: 
                    continue
                pairs.append((center, ids[j]))
    return pairs

pairs = build_pairs(corpus, window=1)

# -------------------------
# 3) negative sampling distribution (unigram^0.75)
# -------------------------
#把每个词出现的次数按id顺序变成tensor
pow_freq = torch.tensor([cnt[id2word[i]] for i in range(V)], dtype=torch.float)
#压平分布，对词频做0.75次幂，高频词概率会下降，低频词概率会上升
neg_dist = pow_freq.pow(0.75)
#归一化概率分布（和为1）
neg_dist = neg_dist / neg_dist.sum()

#按 neg_dist 抽样：
#1.num_samples=batch_size*K：一次性抽出整个 batch 需要的负样本数
#2.replacement=True：允许重复抽到同一个词（这是正常的，word2vec 就是）
#.view(batch_size, K)：型状变成（B,K），每一个正样本配K个负样本num_samples=batch
def sample_negatives(batch_size, K):
    # returns (batch_size, K) negative word ids
    return torch.multinomial(neg_dist, num_samples=batch_size*K, replacement=True).view(batch_size, K)

# -------------------------
# 4) model
# -------------------------
class Word2VecNeg(nn.Module):
    def __init__(self, vocab_size, dim):
        super().__init__()
        #当词作为中心词时用
        self.in_embed = nn.Embedding(vocab_size, dim)   # u_w
        #当词作为上下文词（正/负样本）时用
        self.out_embed = nn.Embedding(vocab_size, dim)  # v_c
        #初始化权重
        #1.计算初始化范围
        initrange = 0.5 / dim
        #2.对中心词向量矩阵进行均匀分布初始化
        self.in_embed.weight.data.uniform_(-initrange, initrange)
        self.out_embed.weight.data.uniform_(-initrange, initrange)


    def forward(self, center_ids, pos_ids, neg_ids):
        # center_ids: (B,)
        # pos_ids:    (B,)
        # neg_ids:    (B, K)

        u = self.in_embed(center_ids)         # (B, D)
        v_pos = self.out_embed(pos_ids)       # (B, D)
        v_neg = self.out_embed(neg_ids)       # (B, K, D)

        # positive score: (B,)
        pos_score = torch.sum(u * v_pos, dim=1)
        pos_loss = F.logsigmoid(pos_score)    # log σ(u·v_pos)

        # negative score: (B, K)
        #torch.bmm做的时batch的矩阵乘
        neg_score = torch.bmm(v_neg, u.unsqueeze(2)).squeeze(2)  # (B,K,D) x (B,D,1) -> (B,K)
        neg_loss = F.logsigmoid(-neg_score).sum(dim=1)           # Σ log σ(-u·v_neg)

        loss = -(pos_loss + neg_loss).mean()
        return loss

# -------------------------
# 5) training loop
# -------------------------
torch.manual_seed(0)
random.shuffle(pairs)

dim = 16
K = 5
model = Word2VecNeg(V, dim)
opt = torch.optim.Adam(model.parameters(), lr=0.05)

batch_size = 8
for epoch in range(200):
    total = 0.0
    random.shuffle(pairs)
    for i in range(0, len(pairs), batch_size):
        batch = pairs[i:i+batch_size]
        center = torch.tensor([x[0] for x in batch], dtype=torch.long)
        pos = torch.tensor([x[1] for x in batch], dtype=torch.long)
        neg = sample_negatives(len(batch), K)

        loss = model(center, pos, neg)
        opt.zero_grad()
        loss.backward()
        opt.step()
        total += loss.item()

    if (epoch + 1) % 50 == 0:
        print(f"epoch {epoch+1}, loss={total:.4f}")

# -------------------------
# 6) get word vectors + cosine similarity demo
# -------------------------
def cosine(a, b):
    return F.cosine_similarity(a.unsqueeze(0), b.unsqueeze(0)).item()

with torch.no_grad():
    W_in = model.in_embed.weight  # (V, D)
    w_ai = word2id["machine"]
    w_deep = word2id["deep"]
    w_nlp = word2id["word"] if "word" in word2id else None

    print("cos(machine, deep) =", cosine(W_in[w_ai], W_in[w_deep]))


epoch 50, loss=10.9880
epoch 100, loss=10.9198
epoch 150, loss=11.4682
epoch 200, loss=8.1949
cos(machine, deep) = 0.9651544094085693


### (4)cbow与skip-gram的区别
| 维度       | **Skip-gram (中心词 -> 上下文)**                        | **CBOW (上下文 -> 中心词)**             |
| :------- | :------------------------------------------------ | :-------------------------------- |
| **输入输出** | 输入 1 个词，预测 k 个词                                   | 输入 k 个词（求和/平均），预测 1 个词            |
| **对生僻词** | **更友好**。因为每个“中心词-上下文”对都是独立的样本，生僻词即便出现次数少，也多次参与更新。 | **较差**。上下文被平均了，生僻词的信息被周围常见词“淹没”了。 |
| **训练速度** | **慢**。一个窗口产生 2k 个训练样本。                            | **快**。一个窗口只产生 1 个训练样本。            |
| **适用场景** | **精细化任务**（大规模语料，对词义要求高）                           | **快速训练**（对词义精度要求不那么极端）            |

### 如何把skip-gram改成cbow
只需修改 forward 的输入处理：
- Skip-gram: 输入 center_ids (Shape: [batch])。

- CBOW: 输入 context_ids_list (Shape: [batch, 2*window])。

In [ ]:
# CBOW forward 伪代码
contexts = self.in_embed(context_ids_list) # [batch, 2*window, dim]
hidden = torch.mean(contexts, dim=1)          # 求平均 -> [batch, dim]
# 后面拿这个 hidden 当作中心向量，去和 u_embedding 做点积预测目标词


In [ ]:
# 1. 计算初始化范围
initrange = 0.5 / dim 

# 2. 对中心词向量矩阵进行均匀分布初始化
self.in_embed.weight.data.uniform_(-initrange, initrange)

# 3. 对上下文词向量矩阵进行均匀分布初始化
self.out_embed.weight.data.uniform_(-initrange, initrange)


### 为什么模型要手动初始化？

数学原理：
* Word2Vec 的核心计算是两个向量的点积：$Score = v_c \cdot u_o = \sum_{i=1}^{d} v_c[i] \times u_o[i]$。
* $d$ 是维度（embed_dim）。如果 $d$ 很大（比如 300），而每个元素 $v_c[i]$ 初始值比较大（比如 1.0），那么 300 个数相加，结果会非常大（比如 300.0）。
* Sigmoid 的灾难：
* 公式是 $\sigma(x) = \frac{1}{1+e^{-x}}$。
* 如果输入 $x$ 是 10 或 300，Sigmoid 输出极其接近 1。
* Sigmoid 的导数是 $\sigma(x)(1-\sigma(x))$。当 $\sigma(x) \approx 1$ 时，导数 $\approx 0$。
* 结果：梯度消失，模型一开始就“死”了，怎么训练都不动。

为什么除以 embed_dim：

为了让点积的结果保持在 0 附近（Sigmoid 的线性区，梯度最大），我们需要让每个元素足够小。

维度 $d$ 越大，累加项越多，所以单个元素必须越小。这是一个经典的缩放技巧（类似 Xavier 初始化思想）。

0.5 是一个经验值（Heuristic），你用 1.0 或 0.1 也可以，重点是 与维度成反比。